In [1]:
!pip install --upgrade google-auth 

Requirement already up-to-date: google-auth in /opt/conda/lib/python3.7/site-packages (1.24.0)


In [2]:
!pip install --upgrade absl-py

Requirement already up-to-date: absl-py in /opt/conda/lib/python3.7/site-packages (0.11.0)


In [3]:
!pip install --upgrade protobuf

Requirement already up-to-date: protobuf in /opt/conda/lib/python3.7/site-packages (3.14.0)


In [4]:
!pip install --upgrade -q tensorflow

In [5]:
# A dependency of the preprocessing for BERT inputs
!pip install -q tensorflow-text

In [6]:
!pip install -q tf-models-official

In [7]:
import tensorflow as tf
import tensorflow_text
import tensorflow_hub as hub

In [8]:
tokenizer = hub.load(
    'https://code.aliyun.com/qhduan/bert_v4/raw/500019068f2c715d4b344c3e2216cef280a7f800/bert_tokenizer_chinese.tar.gz'
)

In [9]:
tokenizer

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7fc29c748450>

In [10]:
albert = hub.load(
    'https://code.aliyun.com/qhduan/bert_v4/raw/500019068f2c715d4b344c3e2216cef280a7f800/albert_tiny.tar.gz'
)

In [11]:
out = albert(tokenizer(['你好']))

In [12]:
out

{'sequence_output': <tf.Tensor: shape=(1, 2, 312), dtype=float32, numpy=
 array([[[ 7.33715475e-01, -7.20572650e-01,  4.15653765e-01,
           5.23570850e-02,  3.98861319e-02,  7.85249993e-02,
           7.95624256e-01, -5.55125475e-01, -1.42330432e+00,
           1.34107754e-01,  2.49635339e-01, -4.13607389e-01,
          -9.09700394e-01, -1.75745285e+00, -1.25018692e+00,
           4.09901619e-01, -2.81636417e-01,  1.46800458e-01,
           3.74855608e-01, -4.27004158e-01,  7.11121559e-01,
          -2.26847202e-01,  5.06151974e-01,  1.00076705e-01,
          -3.53587061e-01,  2.95837283e-01, -3.37562680e-01,
          -9.72694829e-02, -2.91678369e-01, -1.17791247e+00,
           1.06728113e+00,  6.04827881e-01,  2.32241750e-01,
           8.05636644e-01,  1.79337010e-01,  1.16546011e+00,
          -1.73346031e+00, -1.70589745e-01, -1.84808180e-01,
          -6.19936347e-01,  1.57373667e-01,  6.38421416e-01,
           5.00105202e-01,  4.56167281e-01,  2.58985817e-01,
          -5

In [13]:
#assert out['sequence_output'].shape == (1, 2, 312)

### BERT 分類模型

In [14]:
x = [
    ['天氣好'],
    ['天氣差'],
    ['下雨'],
    ['晴天'],
    ['雨'],
    ['晴'],
]
y = [
    1, 0, 0, 1, 0, 1
]

In [15]:
x,y

([['天氣好'], ['天氣差'], ['下雨'], ['晴天'], ['雨'], ['晴']], [1, 0, 0, 1, 0, 1])

In [16]:
tx = tf.constant(x)
ty = tf.constant(tf.keras.utils.to_categorical(y, 2))

In [17]:
tx,ty

(<tf.Tensor: shape=(6, 1), dtype=string, numpy=
 array([[b'\xe5\xa4\xa9\xe6\xb0\xa3\xe5\xa5\xbd'],
        [b'\xe5\xa4\xa9\xe6\xb0\xa3\xe5\xb7\xae'],
        [b'\xe4\xb8\x8b\xe9\x9b\xa8'],
        [b'\xe6\x99\xb4\xe5\xa4\xa9'],
        [b'\xe9\x9b\xa8'],
        [b'\xe6\x99\xb4']], dtype=object)>,
 <tf.Tensor: shape=(6, 2), dtype=float32, numpy=
 array([[0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.]], dtype=float32)>)

In [18]:
bert = hub.KerasLayer(
            'https://code.aliyun.com/qhduan/bert_v4/raw/500019068f2c715d4b344c3e2216cef280a7f800/albert_large.tar.gz', 
            trainable=True,
            output_key='pooled_output'
            )

In [19]:
inputs = tf.keras.layers.Input(shape=(None,), dtype=tf.string)

In [20]:
m = inputs

In [21]:
m = bert(inputs)

In [22]:
m = tf.keras.layers.Masking()(m)

In [23]:
#m = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, ))(m)

In [24]:
m = tf.keras.layers.Dense(2, activation='softmax')(m)

In [25]:
model = tf.keras.Model(inputs=inputs, outputs=m)

In [26]:
model.compile(loss='categorical_crossentropy')

In [ ]:
model.fit(tx, ty, epochs=10, batch_size=2)

Epoch 1/10


In [ ]:
logits = model.predict(tx)

In [ ]:
pred = logits.argmax(-1).tolist()

In [ ]:
pred, y